<a href="https://colab.research.google.com/github/MuskanSinhaPi/Intern/blob/main/STHSL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf STHSL

In [ ]:
!git clone https://github.com/MuskanSinhaPi/STHSL

Cloning into 'STHSL'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 162 (delta 77), reused 81 (delta 38), pack-reused 9
Receiving objects: 100% (162/162), 7.08 MiB | 32.64 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [ ]:
%cd STHSL

/content/STHSL


##Data Preprocessing for SF Crime Dataset

In [ ]:
import time
import numpy as np
import pickle

offenseMap = {'Violent Crimes': 0, 'Property Crimes': 1, 'Miscellaneous': 2, 'Administrative and Other Offenses': 3, 'Drug and Alcohol Related': 4, 'Non-Criminal and Special Cases': 5, 'Public Order Crimes': 6, 'White Collar Crimes': 7}
offenseSet = set()
latSet = set()
lonSet = set()
timeSet = set()
data = []
with open('Datasets/SF_crime/SFCrime.csv', 'r') as fs:
	fs.readline()
	for line in fs:
		arr = line.strip().split(',')
		print(arr)

		timeArray = time.strptime(arr[0], '%m/%d/%Y %I:%M:%S %p')
		timestamp = time.mktime(timeArray)
		offense = offenseMap[arr[1]]
		lat = float(arr[2])
		lon = float(arr[3])

		latSet.add(lat)
		lonSet.add(lon)
		timeSet.add(timestamp)
		offenseSet.add(offense)

		data.append({
			'time': timestamp,
			'offense': offense,
			'lat': lat,
			'lon': lon
		})
print('Length of data', len(data), '\n')
print('Offense:', offenseSet, '\n')
print('Latitude:', min(latSet), max(latSet))
print('Longtitude:', min(lonSet), max(lonSet))
print('Latitude:', min(latSet), max(latSet), (max(latSet) - min(latSet)) / (1 / 111), '\n')
print('Longtitude:', min(lonSet), max(lonSet), (max(lonSet) - min(lonSet)) / (1 / 84), '\n')
print('Time:')
minTime = min(timeSet)
maxTime = max(timeSet)
print(time.localtime(minTime))
print(time.localtime(maxTime))

minLat = min(latSet)
minLon = min(lonSet)
maxLat = max(latSet)
maxLon = max(lonSet)
latDiv = 111 / 3 #1
lonDiv = 84 / 3 #1
latNum = int((maxLat - minLat) * latDiv) + 1
lonNum = int((maxLon - minLon) * lonDiv) + 1
trnTensor = np.zeros((latNum, lonNum, 366+365-92-30, len(offenseSet)))
valTensor = np.zeros((latNum, lonNum, 30, len(offenseSet)))
tstTensor = np.zeros((latNum, lonNum, 92, len(offenseSet)))
for i in range(len(data)):
    tup = data[i]
    temT = time.localtime(tup['time'])
    if temT.tm_year == 2016 or temT.tm_year == 2017 and temT.tm_mon < 9:
        day = temT.tm_yday + (0 if temT.tm_year == 2016 else 366) - 1
        row = int((tup['lat'] - minLat) * latDiv)
        col = int((tup['lon'] - minLon) * lonDiv)
        offense = tup['offense']
        trnTensor[row][col][day][offense] += 1
    elif temT.tm_year == 2017 and temT.tm_mon == 9:
        day = temT.tm_mday - 1
        row = int((tup['lat'] - minLat) * latDiv)
        col = int((tup['lon'] - minLon) * lonDiv)
        offense = tup['offense']
        valTensor[row][col][day][offense] += 1
    elif temT.tm_year == 2017 and temT.tm_mon > 9:
        day = temT.tm_yday - (365 - 92) - 1
        row = int((tup['lat'] - minLat) * latDiv)
        col = int((tup['lon'] - minLon) * lonDiv)
        offense = tup['offense']
        tstTensor[row][col][day][offense] += 1

names = ['trn.pkl', 'val.pkl', 'tst.pkl']
tensors = [trnTensor, valTensor, tstTensor]
for i in range(len(names)):
	with open('Datasets/SF_crime/' + names[i], 'wb') as fs:
		pickle.dump(tensors[i], fs)

Streaming output truncated to the last 5000 lines.
['11/18/2015 02:30:00 PM', 'Administrative and Other Offenses', '-122.47114555295', '37.7638796121694']
['05/13/2010 11:00:00 AM', 'Drug and Alcohol Related', '-122.399443658942', '37.7301760161647']
['04/06/2015 03:45:00 PM', 'Property Crimes', '-122.394565756376', '37.7370657700508']
['01/15/2018 07:05:00 PM', 'Administrative and Other Offenses', '-122.47564746078616', '37.72852806274646']
['01/04/2015 12:30:00 PM', 'Property Crimes', '-122.407702149718', '37.7706967930865']
['01/25/2015 11:00:00 AM', 'Property Crimes', '-122.38792119559', '37.7643790292129']
['09/24/2017 11:30:00 PM', 'Property Crimes', '-122.40359529351431', '37.72763409925056']
['11/30/2015 02:00:00 AM', 'Non-Criminal and Special Cases', '-122.456583021277', '37.7258073001782']
['06/01/2015 03:00:00 PM', 'Non-Criminal and Special Cases', '-122.431957685196', '37.7649357189679']
['05/26/2015 10:00:00 AM', 'Property Crimes', '-122.411156854389', '37.8059786528897']


In [ ]:
# import time
# import numpy as np
# import pickle

# offenseMap = {'Violent Crimes': 0, 'Property Crimes': 1, 'Miscellaneous': 2, 'Administrative and Other Offenses': 3, 'Drug and Alcohol Related': 4, 'Non-Criminal and Special Cases': 5, 'Public Order Crimes': 6, 'White Collar Crimes': 7}
# offenseSet = set()
# latSet = set()
# lonSet = set()
# timeSet = set()
# data = []
# with open('Datasets/SF_crime/SFCrime.csv', 'r') as fs:
# 	fs.readline()
# 	for line in fs:
# 		arr = line.strip().split(',')
# 		timeArray = time.strptime(arr[0], '%m/%d/%Y %I:%M:%S %p')
# 		timestamp = time.mktime(timeArray)
# 		offense = offenseMap[arr[1]]
# 		lat = float(arr[2])
# 		lon = float(arr[3])

# 		latSet.add(lat)
# 		lonSet.add(lon)
# 		timeSet.add(timestamp)
# 		offenseSet.add(offense)

# 		data.append({
# 			'time': timestamp,
# 			'offense': offense,
# 			'lat': lat,
# 			'lon': lon
# 		})

# minTime = min(timeSet)
# maxTime = max(timeSet)
# minLat = min(latSet)
# minLon = min(lonSet)
# maxLat = max(latSet)
# maxLon = max(lonSet)
# latDiv = 111 / 32 #1
# lonDiv = 84 / 32 #1
# latNum = int((maxLat - minLat) * latDiv) + 1
# lonNum = int((maxLon - minLon) * lonDiv) + 1
# trnTensor = np.zeros((latNum, lonNum, 366+365-92-30, len(offenseSet)))
# valTensor = np.zeros((latNum, lonNum, 30, len(offenseSet)))
# tstTensor = np.zeros((latNum, lonNum, 92, len(offenseSet)))
# for i in range(len(data)):
#   day=0
#   tup = data[i]
#   temT = time.localtime(tup['time'])
#   if temT.tm_year == 2016 or temT.tm_year == 2017 and temT.tm_mon < 9:
# 		day = temT.tm_yday + (0 if temT.tm_year == 2016 else 366) - 1
#     row = int((tup['lat'] - minLat) * latDiv)
#     col = int((tup['lon'] - minLon) * lonDiv)
#     offense = tup['offense']
#     trnTensor[row][col][day][offense] += 1
#   elif temT.tm_year == 2017 and temT.tm_mon == 9:
#     day = temT.tm_mday - 1
#     row = int((tup['lat'] - minLat) * latDiv)
#     col = int((tup['lon'] - minLon) * lonDiv)
#     offense = tup['offense']
#     valTensor[row][col][day][offense] += 1
#   elif temT.tm_year == 2017 and temT.tm_mon > 9:
# 		day = temT.tm_yday - (365 - 92) - 1
#     row = int((tup['lat'] - minLat) * latDiv)
#     col = int((tup['lon'] - minLon) * lonDiv)
#     offense = tup['offense']
#     tstTensor[row][col][day][offense] += 1
#   print(f"row: {row}, col: {col}, day: {day}, offense: {offense}")
#   # tensor[row][col][day][offense] += 1

# names = ['trn.pkl', 'val.pkl', 'tst.pkl']
# tensors = [trnTensor, valTensor, tstTensor]
# for i in range(len(names)):
# 	with open('Datasets/SF_crime/' + names[i], 'wb') as fs:
# 		pickle.dump(tensors[i], fs)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 52)

In [ ]:
!python /content/STHSL/train.py --data NYC

self.predir Datasets/NYC_crime/
(966, 92, 8)
Row: 7 , Col: 138
Sparsity: 0.00022416512832019366
start training...
Traceback (most recent call last):
  File "/content/STHSL/train.py", line 51, in <module>
    main()
  File "/content/STHSL/train.py", line 21, in main
    metrics, metrics1 = engine.train()
  File "/content/STHSL/engine.py", line 84, in train
    out_local, eb_local, eb_global, Infomax_pred, out_global = self.model(feats, DGI_feats)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return forward_call(*args, **kwargs)
  File "/content/STHSL/model.py", line 177, in forward
    hy_embeds, Infomax_pred = self.Hypergraph_Infomax(embeds_in_global, DGI_neg)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    ret